In [39]:
import tweepy
from datetime import datetime

In [56]:
# !pip install textblob

In [58]:
# !pip install vaderSentiment

In [61]:
# !pip install unidecode

In [103]:
import json
from tweepy import OAuthHandler
from tweepy import Stream
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from elasticsearch import Elasticsearch
from datetime import datetime
from unidecode import unidecode
import calendar
import numpy as np
from http.client import IncompleteRead
from urllib3.exceptions import ProtocolError
from queue import Queue
from threading import Thread
import pandas as pd

In [34]:
consumerKey = "0T8qnnU1X2KpMjlruLjWEeOQo"

consumerSecret = "a6aeHMJHumDJL86fOQxdBkSLxTRjT56OD0RnACi2yc62TVAY1V"

accessToken = "1603044224-CWe5dp81Z4SsP3DDR1LuSulJgMuoYulqhaRbCjK"

accessTokenSecret = "0UkRFMAGhwGZRaJ5SzKJ0v8gAekB1Msrx4q5Ot0ZcC0c4"

In [26]:
tweets = api.search_tweets(q = 'jpmorgan', count = 1000)

In [27]:
tweets_list = []

In [28]:
for tweet in tweets:

    print(tweet.text)

    if tweet in tweets_list:
        deadend = True
        break
    tweets_list += [tweet]

RT @Tony73_censored: Acusadores de Jeffrey Epstein demandan a Deutsche Bank y JPMorgan.

Las demandas afirman que los bancos facilitaron el…
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and…
RT @DaveCBenoit: Victims of Jeffrey Epstein abuse sued JPMorgan and Deutsche Bank overnight, saying they facilitated his alleged sex-traffi…
RT @dragonadvantage: JPMorgan Reportedly Giving 16 Weeks Paid Leave to All New Parents After Settling $5 Million 'Anti Dad' Bias Case https…
RT @daroHC: El CEO de JPMorgan, Jamie Dimon, visitó  Uruguay, donde se reunió con el presidente Luis Lacalle Pou y, en encuentros separados…
@jpmorgan #wallet #exchange #NEWS 
JP MORGAN prepare your own wallet and digital exchange, and then some say that c… https://t.co/jnnrBueyIt
RT @daroHC: El CEO de JPMorgan, Jamie Dimon, visitó  Uruguay, donde se reunió con el presidente Luis Lacalle Pou y, en encuentros separados…
"While at JPMo

In [29]:
tweets_list

[Status(_api=<tweepy.api.API object at 0x000001678DB3C670>, _json={'created_at': 'Sun Nov 27 21:56:31 +0000 2022', 'id': 1596986353107058688, 'id_str': '1596986353107058688', 'text': 'RT @Tony73_censored: Acusadores de Jeffrey Epstein demandan a Deutsche Bank y JPMorgan.\n\nLas demandas afirman que los bancos facilitaron el…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'Tony73_censored', 'name': 'Tony Aberturas San José', 'id': 1453005729875566599, 'id_str': '1453005729875566599', 'indices': [3, 19]}], 'urls': []}, 'metadata': {'iso_language_code': 'es', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1277670952214704128, 'id_str': '1277670952214704128', 'name': 'maria', 'screen_name'

In [30]:
tweets_list[1]._json

{'created_at': 'Sun Nov 27 21:55:48 +0000 2022',
 'id': 1596986170776264706,
 'id_str': '1596986170776264706',
 'text': 'RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'Phil_Lewis_',
    'name': 'philip lewis',
    'id': 286998245,
    'id_str': '286998245',
    'indices': [3, 15]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1224003416512315393,
  'id_str': '1224003416512315393',
  'name': 'Hyperborean',
  'screen_name': 'HyperboreanK1ng',
  'location': 'Valhalla, NY',
  'descr

In [ ]:
for data in tweets_:
    tweet = data._json['text']
    vs = analyzer.polarity_scores(tweet)
    score = vs['compound']
    if score:

        if score < 0:
            sentiment = "negative"
        elif score == 0:
            sentiment = "neutral"
        else:
            sentiment = "positive"
        print(tweet, score, sentiment)

        # extract the first hashtag from the object
        # transform the Hashtags into proper case
        if len(data._json["entities"]["hashtags"])>0:
            hashtags=data._json["entities"]["hashtags"][0]["text"].title()
        else:
            #Elasticeach does not take None object
            hashtags="None"
            
        es.index(index="world_bank_fintech",
                 doc_type="twitter2",
                 body={"author": data._json["user"]["screen_name"],
                       "followers":data._json["user"]["followers_count"],
                       "date": datetime.strptime(data._json["created_at"], '%a %b %d %H:%M:%S %z %Y'),
                       "message": data._json["text"]  if "text" in data._json.keys() else " ",
                       "hashtags": hashtags,
                       "score": score,
                       "retweet": data._json["retweet_count"],
                       "location": data._json["user"]["location"],
                       "sentiment": sentiment})

In [37]:
data = tweets_list[1]

In [44]:
body ={"author": data._json["user"]["screen_name"],
       "followers":data._json["user"]["followers_count"],
       "date": datetime.strptime(data._json["created_at"], '%a %b %d %H:%M:%S %z %Y'),
       "message": data._json["text"]  if "text" in data._json.keys() else " ",
       "retweet": data._json["retweet_count"],
       "location": data._json["user"]["location"]}

In [45]:
body

{'author': 'HyperboreanK1ng',
 'followers': 7,
 'date': datetime.datetime(2022, 11, 27, 21, 55, 48, tzinfo=datetime.timezone.utc),
 'message': 'RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and…',
 'retweet': 3653,
 'location': 'Valhalla, NY'}

In [88]:
data_dict = {}

In [89]:
company_handle_list = {'jpmorgan':'JPMorgan Chase & Co.', 'Mastercard':'Mastercard', 'HSBC Holdings PLC':'HSBC', 
                       'Lloyds Banking Group':'LloydsBank', 'Barclays':'Barclays', 'American International Group':'AIGinsurance',
                       'Canadian Imperial Bank of Commerce':'cibc', 'American Express':'AmericanExpress', 
                       'The Goldman Sachs Group':'GoldmanSachs', 'Bank of America':'BankofAmerica', 'VMware':'VMware', 
                       'Atlassian':'Atlassian', 'Amazon':'amazon', 'Apple':'Apple', 'Broadcom Inc.':'Broadcom',
                       'Adobe':'Adobe', 'Nokia':'nokia', 'Dell':'Dell', 'Autodesk':'autodesk', 'Shopify':'Shopify',
                       'Duke Energy Corporation':'DukeEnergy', 'DTE Energy Company':'DTE_Energy', 'Shell':'Shell',
                       'Exxon Mobil':'exxonmobil', 'BP':'bp_plc', 'Hess Corporation':'HessCorporation',
                       'Marathon Oil Corporation':'MarathonOil', 'Ecopetrol S.A.':'ECOPETROL_SA',
                       'PetroChina Co. Ltd.':'Petrochina_Iraq', 'PetroChina Co. Ltd.':'PetroChina', 'Schlumberger':'Schlumberger'}

In [90]:
# company_handle_list = {'jpmorgan':'JPMorgan Chase & Co.', 'Mastercard':'Mastercard'}

In [91]:
sector_list = ["Financial Services", "Financial Services", "Financial Services", "Financial Services", "Financial Services", 
              "Financial Services", "Financial Services", "Financial Services", "Financial Services", "Financial Services", 
              "Technology", "Technology", "Technology", "Technology", "Technology", "Technology", "Technology", "Technology", 
              "Technology", "Technology", "Energy", "Energy", "Energy", "Energy", "Energy", "Energy", "Energy", "Energy", 
              "Energy", "Energy"]

In [124]:
handle_list = list(company_handle_list.keys())

In [125]:
sector_map = {handle_list[i]:sector_list[i] for i in range(len(handle_list))}

In [126]:
sector_map

{'jpmorgan': 'Financial Services',
 'Mastercard': 'Financial Services',
 'HSBC Holdings PLC': 'Financial Services',
 'Lloyds Banking Group': 'Financial Services',
 'Barclays': 'Financial Services',
 'American International Group': 'Financial Services',
 'Canadian Imperial Bank of Commerce': 'Financial Services',
 'American Express': 'Financial Services',
 'The Goldman Sachs Group': 'Financial Services',
 'Bank of America': 'Financial Services',
 'VMware': 'Technology',
 'Atlassian': 'Technology',
 'Amazon': 'Technology',
 'Apple': 'Technology',
 'Broadcom Inc.': 'Technology',
 'Adobe': 'Technology',
 'Nokia': 'Technology',
 'Dell': 'Technology',
 'Autodesk': 'Technology',
 'Shopify': 'Technology',
 'Duke Energy Corporation': 'Energy',
 'DTE Energy Company': 'Energy',
 'Shell': 'Energy',
 'Exxon Mobil': 'Energy',
 'BP': 'Energy',
 'Hess Corporation': 'Energy',
 'Marathon Oil Corporation': 'Energy',
 'Ecopetrol S.A.': 'Energy',
 'PetroChina Co. Ltd.': 'Energy',
 'Schlumberger': 'Energy'}

In [92]:
analyzer = SentimentIntensityAnalyzer()

In [99]:
for query, company_name in company_handle_list.items():
    tweets = api.search_tweets(q = query, count = 1000)
    
    
    tweets_list = []
    for data in tweets:
        tweet = data._json['text']
        vs = analyzer.polarity_scores(tweet)
        score = vs['compound']

        if score:

            if score < 0:
                sentiment = "negative"
            elif score == 0:
                sentiment = "neutral"
            else:
                sentiment = "positive"
            print(tweet, score, sentiment)

            # extract the first hashtag from the object
            # transform the Hashtags into proper case
            if len(data._json["entities"]["hashtags"])>0:
                hashtags=data._json["entities"]["hashtags"][0]["text"].title()
            else:
                #Elasticeach does not take None object
                hashtags="None"
                
            
            try:
                author = data._json["user"]["screen_name"]
            except:
                author = "unknown"

            try:
                followers = data._json["user"]["followers_count"]
            except:
                followers = np.NaN
                
            try:
                message = data._json["text"]  if "text" in data._json.keys() else " "
            except:
                message = ""

            try:
                retweet = data._json["retweet_count"]
            except:
                retweet = np.NaN
                
            try:
                popularity_score = data._json["retweet_count"]+data._json["favourites_count"]
            except:
                popularity_score = 0

            try:
                reach_score = data._json["user"]["followers_count"]-data._json["user"]["friends_count"]
            except:
                reach_score = 0
                
            body={"author":author,
                  "followers":followers,
                  "date":datetime.strptime(data._json["created_at"], '%a %b %d %H:%M:%S %z %Y'),
                  "message":message,
                  "hashtags": hashtags,
                  "retweet":retweet,
                  "location": data._json["user"]["location"],
                  "score": score,
                  "sentiment": sentiment,
                  "popularity score":popularity_score,
                  "reach score":reach_score,
                  "relevance score":data._json["user"]["statuses_count"]+popularity_score+reach_score}

            tweets_list += [body]
        
    data_dict[company_name] = tweets_list

RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
RT @elizableu: Survivors of Jeffrey Epstein filed lawsuits against Deutsche Bank AG and JPMorgan Chase &amp; Co today. 

The banks financially… -0.1531 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
RT @Loveon999: 4/ The accusations against JPMorgan lean heavily on the close relationship between Staley and Epstein, his private banker at… -0.3182 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank hav

RT @tideexchange: When abroad you can pay in the local currency or native facilitated by Visa / Mastercard

@LayerZero_Labshas has enabled… -0.1027 negative
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 V

RT @TheBlackWiseGuy: @NedBeale @ixshopping @financialombuds This corrupt fraudulent woman refused to answer questions on her subordinates o… -0.6597 negative
RT @TheBlackWiseGuy: @NedBeale @ixshopping @financialombuds This corrupt fraudulent woman refused to answer questions on her subordinates o… -0.6597 negative
RT @turn2us_org: We’ve seen a sharp increase in demand of our services, especially with the #CostOfLiving crisis. That’s why partnerships w… -0.5106 negative
RT @TheBlackWiseGuy: This fraudulent disease ridden corrupt woman has supported staff who have knowingly gave false adjudications against a… -0.3506 negative
RT @TheBlackWiseGuy: @NedBeale @ixshopping @financialombuds This corrupt fraudulent woman refused to answer questions on her subordinates o… -0.6597 negative
RT @smepathfinder: Watch out for the insolvency predators working for Lloyds Banking Group and RBS NatWest? CBILs and BBILS victims are on… -0.3182 negative
This fraudulent disease ridden corrupt woman has supp

Just to mention some of the world bigger scandals in multiple industries and all of them with very strict regulatio… https://t.co/psEhkltcJL -0.4939 negative
RT @n194at: American Trans Air (ATA Airlines)
Lockheed L1011-500 N164AT
BUD/LHBP Budapest Ferihegy International Airport
Photo credit Feren… 0.3818 positive
Another shock result!  

American gun right group Safari Club International - which has shelled out £11 MILLION a Y… https://t.co/oxrPX8W27J -0.6476 negative
RT @JorgeDBL: Havana Consulting Group, Akerman, Akin Gump, Albright Stonebridge Group, American Maket Intelligence, Ankura Consulting, Carl… 0.4767 positive
RT @JorgeDBL: Havana Consulting Group, Akerman, Akin Gump, Albright Stonebridge Group, American Maket Intelligence, Ankura Consulting, Carl… 0.4767 positive
BLM, specifically the American chapter, not the international chapters that were duped, is nothing but a group of M… https://t.co/A0dalHDyRs -0.2263 negative
RT @FoundationZalmi: The group will spend the day at th

RT @romoolo: Network Status Today
— — — — — — — — — — — 
Federal Reserve    Offline 🔴
NYSE                       Offline🔴
JP Morgan… 0.1363 positive
RT @biokript: Don't let the idea of Proof of Reserve make you think centralized exchanges can be trusted with your money. 

#biokript #noty… 0.4767 positive
Bank of America To Partner With Ripple for ODL Services After XRP Lawsuit Finishes! 👀
https://t.co/igY3khoAxw -0.2942 negative
RT @WatcherGuru: Bank of America says #Ethereum’s merge to proof-of-stake may increase institutional adoption. 0.3182 positive
RT @itsloganjones: Buy Companies that you use everyday 

☕️ $SBUX Starbucks
🧼 $PG Proctor &amp; Gamble 
💳 $V Visa 
📲 $AAPL Apple 
🛒 $WMT Walmar… 0.3818 positive
Article: Mercy Home Heroes | Thomas’ Fundraising Page : 2023 Bank of America... https://t.co/naru2Mm3Yr on @bloglovin 0.7003 positive
RT @MemeingBitcoin: Network Status Nov. 27th
— — — — — — — — — — 
Federal Reserve    Offline 🔴
NYSE                       Offline🔴
JP Morga… 0.13

...

RT @libsoftiktok: Shopify CEO responds to pressure from the media to ban my shopify account 👑 https://t.co/4hmFzkZaoj -0.7003 negative
@gregg_re @libsoftiktok @BarrettWilson6 @elonmusk @TunneyTun @CBC @Shopify Sick, just sick -0.765 negative
RT @libsoftiktok: Shopify CEO responds to pressure from the media to ban my shopify account 👑 https://t.co/4hmFzkZaoj -0.7003 negative
@MrBeast @Shopify I Want Job Sir. I am Very Frustrated In Life Sir. I Have Lots Of New Ideas Of Business But I Have… https://t.co/8zm1fyyUXk -0.2952 negative
RT @libsoftiktok: Shopify CEO responds to pressure from the media to ban my shopify account 👑 https://t.co/4hmFzkZaoj -0.7003 negative
God give me strength to accept the things I cannot change, the courage to change the things I can, &amp; the wisdom to… https://t.co/Zd0oy9DOLd 0.8258 positive
RT @libsoftiktok: Shopify CEO responds to pressure from the media to ban my shopify account 👑 https://t.co/4hmFzkZaoj -0.7003 negative
@taliaotg @nandoodles @Shopify Why 

RT @SreeIyer1: DHFL Rs.34,615 cr loan fraud case: Money laundered via 87 shell firms, created 2,60,000 fake borrowers - https://t.co/vgqTtB… -0.7096 negative
@RonO__RNS Them Uk Niggas always say “ Know what I’m sayin?” No nigga lmao we don’t Lmao 🤣 wtf . Sound like he tryi… https://t.co/FNf7Hz9EY3 -0.2373 negative
ALX SE second phase returns today 
After barely escaping the gates of shell, i now have to survive mighty python 🫠 -0.024 negative
RT @Kunal_Biswas707: Indian Army Live Fire Of M982A1 Excalibur Precision Shell With Maximum Range Of 50 Kilometers Costs 60 Lakh Each And I… -0.34 negative
RT @DhanushSlave: Happy Birthday @Gowtham_Soupboy macha 🫂💙🎉 
Shell God bless you with Forever happiness and success and good Health 🥰🤗

#Va… 0.988 positive
You wake up feeling innocent. You taste shell four feet away. You have 21 talons,  46 scales, and  50 scales. You h… https://t.co/0JEOn6dbua 0.4404 positive
@Safid_Deen You think? Didn't aj get hurt too? Dam we might have to play against san

RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
Exxon, Mobil Chevron, Amazon, Home Depot, Kroger…have had more to do with price increases https://t.co/E4Ri0SzfY0 0.1779 positive
RT @AngelRe96386095: Folks, Exxon Mobil reported $17B in profits last quarter alone.
Just wanted to start your day out right. 👠 0.2263 positive
Baby ceiling tile installer commits wage theft at Exxon Mobil over losing iPad -0.3612 negative
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
Tell that to Exxon Mobil, Shell, BP. No one is leashing them. They are price-gouging. Biden has begged them to step… https://t.co/51XHI9fIzj -0.296 negative
RT

Finally reached level 50 BP RTD when/j
#genshintwt https://t.co/AS5JKBObVF 0.1027 positive
RT @prim813: .⋅ ۵♡۵ ⋅. BP HYMM GA .⋅ ۵♡۵ ⋅.
- one winner of bp hymm 
- like retweet and follow @prim813 
- will end on december 2nd .⋅ ۵♡۵… 0.743 positive
RT @prim813: .⋅ ۵♡۵ ⋅. BP HYMM GA .⋅ ۵♡۵ ⋅.
- one winner of bp hymm 
- like retweet and follow @prim813 
- will end on december 2nd .⋅ ۵♡۵… 0.743 positive
@weirdku_catfox @BP_Xardon Wtf que paja eso ah encima siemore para esa ref? Pq si es eso twitter le hace fullying a tu pokesona noOOoO (? -0.6322 negative
RT @bb0un_official: โอ้ะ! อันดับ4แน้ว ยอดสะสม 259K มันเฉียดๆไหมนะ👀 แต่กองกำลังเก่งมากต่ะ ยังไหวกันไหมนะ!🎉🫶🏻

 #BetweenUsEP4 https://t.co/Kd… 0.508 positive
RT @trendforprem: รักแค่ไหนไปให้สุด อยากเห็น 300k สวยๆประทับแท็กเชือกป่านค้าบบ✨

#BetweenUsEP4 0.3182 positive
@fantasylisa05 The fact that BP doesn't arrange concert or arrange small concerts where they famous in like India,… https://t.co/0d4SMfLesm 0.3612 positive
RT @Elkrosmediahub: P

Hess Corporation, we need green infrastructure desperately! How will you help our communities? #climatejustice 0.1007 positive
Devon Energy Corporation : Goldman Sachs se mantiene neutral.
DVN

Pioneer Natural Resources Company : El Goldman S… https://t.co/mvff9mxWza 0.5574 positive
Hess Corporation's quarterly revenue growth rate YoY of 77.50% ranked 456 out of 5765 companies in our database.… https://t.co/BXmFVTrlnr 0.3818 positive
Hess Corporation $HES quarterly revenue growth rate YoY of 77.50% ranked

⭐⭐⭐⭐⭐

in our db. Impressive! https://t.co/SVOpS7QHYl 0.7345 positive
A Strong Upward Move for HESS Corporation - https://t.co/HjRrCi9rYf https://t.co/gIzOkFouNn 0.5106 positive
Fitch Ratings maintains Ecopetrol S.A.'s rating at BB , with a stable outlook https://t.co/sCkCQdiSZy 0.296 positive
#Fiestas ||Empresas petroleras como ECOPETROL S.A y Frontera Energy a través de un comunicado desmienten patrocinio… https://t.co/rTzYqdpDkb 0.5574 positive
SKK Migas reveals that Chinese oil a

In [100]:
data_dict

{'Schlumberger': [{'author': 'israel_amara',
   'followers': 25,
   'date': datetime.datetime(2022, 11, 28, 4, 19, 55, tzinfo=datetime.timezone.utc),
   'message': 'zaman pirler bulunmuyor salonlarda. Her gun bir M. de bileskeden kurtulur gibi taninmasini engelleyen cehreden ve… https://t.co/VXoHH2f8Yo',
   'hashtags': 'None',
   'retweet': 45,
   'location': '',
   'score': -0.34,
   'sentiment': 'negative',
   'popularity score': 0,
   'reach score': -17,
   'relevance score': 882},
  {'author': 'musproest',
   'followers': 412,
   'date': datetime.datetime(2022, 11, 28, 4, 6, 2, tzinfo=datetime.timezone.utc),
   'message': 'Sin ir tan lejos, en el Ilustre Instituto llegan cada año los de Schlumberger y Halliburton (por mencionar algunas)… https://t.co/dx1lJ9OqfC',
   'hashtags': 'None',
   'retweet': 0,
   'location': 'Mérida, Yucatán',
   'score': -0.5574,
   'sentiment': 'negative',
   'popularity score': 0,
   'reach score': 206,
   'relevance score': 38380}],
 'JPMorgan Chase & 

In [101]:
data_dict.keys()

dict_keys(['Schlumberger', 'JPMorgan Chase & Co.', 'Mastercard', 'HSBC', 'LloydsBank', 'Barclays', 'AIGinsurance', 'cibc', 'AmericanExpress', 'GoldmanSachs', 'BankofAmerica', 'VMware', 'Atlassian', 'amazon', 'Apple', 'Broadcom', 'Adobe', 'nokia', 'Dell', 'autodesk', 'Shopify', 'DukeEnergy', 'DTE_Energy', 'Shell', 'exxonmobil', 'bp_plc', 'HessCorporation', 'MarathonOil', 'ECOPETROL_SA', 'PetroChina'])

In [106]:
pd.DataFrame(data_dict['JPMorgan Chase & Co.'])

,author,followers,date,message,hashtags,retweet,location,score,sentiment,popularity score,reach score,relevance score
0,mvelo_dlamini,1857,2022-11-28 07:15:15+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,"London, England",-0.5859,negative,0,154,25698
1,2njtd65rhx,113,2022-11-28 07:14:41+00:00,RT @elizableu: Survivors of Jeffrey Epstein fi...,None,3189,,-0.1531,negative,0,-577,1257
2,wellnessFR,3100,2022-11-28 07:11:21+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,Europe,-0.5859,negative,0,-1798,181283
3,JoviLeMonstre,58923,2022-11-28 07:08:51+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,Cameroon 🇨🇲,-0.5859,negative,0,58850,66600
4,chad_oliphant,161,2022-11-28 07:05:05+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,,-0.5859,negative,0,-786,1339
5,SusanChubb1,8994,2022-11-28 07:01:31+00:00,RT @Loveon999: 4/ The accusations against JPMo...,None,6,,-0.3182,negative,0,-825,381818
6,LosentHim,217,2022-11-28 07:01:19+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,Lost in the World,-0.5859,negative,0,-418,1140
7,JoelWotshowadi1,272,2022-11-28 06:59:57+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,Belgique,-0.5859,negative,0,57,2597
8,saritaleonn,134,2022-11-28 06:58:01+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4212,"San Francisco, CA",-0.5859,negative,0,-65,5271
9,norfolkgirly,1714,2022-11-28 06:56:57+00:00,RT @chicchan: JPMorgan Chase and Deutsche Bank...,None,17,Norfolk England,-0.5859,negative,0,-554,56444


In [147]:
tweets_list5 = []

In [148]:
for query, company_name in company_handle_list.items():
    tweets = api.search_tweets(q = query, count = 5000)
    
    
    for data in tweets:
        tweet = data._json['text']
        vs = analyzer.polarity_scores(tweet)
        score = vs['compound']

        if score:

            if score < 0:
                sentiment = "negative"
            elif score == 0:
                sentiment = "neutral"
            else:
                sentiment = "positive"
            print(tweet, score, sentiment)

            # extract the first hashtag from the object
            # transform the Hashtags into proper case
            if len(data._json["entities"]["hashtags"])>0:
                hashtags=data._json["entities"]["hashtags"][0]["text"].title()
            else:
                #Elasticeach does not take None object
                hashtags="None"
                
            
            try:
                author = data._json["user"]["screen_name"]
            except:
                author = "unknown"

            try:
                followers = data._json["user"]["followers_count"]
            except:
                followers = np.NaN
                
            try:
                message = data._json["text"]  if "text" in data._json.keys() else " "
            except:
                message = ""

            try:
                retweet = data._json["retweet_count"]
            except:
                retweet = np.NaN
                
            try:
                popularity_score = data._json["retweet_count"]+data._json["user"]["favourites_count"]
            except:
                popularity_score = 0
                
            try:
                likes = data._json["user"]["favourites_count"]
            except:
                likes = np.NaN

            try:
                reach_score = data._json["user"]["followers_count"]-data._json["user"]["friends_count"]
            except:
                reach_score = 0
    
            try:
                statuses_count = data._json["user"]["statuses_count"]
            except:
                statuses_count = 0
                
            body={"author":author,
                  "followers":followers,
                  "date":datetime.strptime(data._json["created_at"], '%a %b %d %H:%M:%S %z %Y'),
                  "message":message,
                  "hashtags": hashtags,
                  "retweet":retweet,
                  "location": data._json["user"]["location"],
                  "score": score,
                  "sentiment": sentiment,
                  "likes":likes,
                  "popularity score":popularity_score,
                  "reach score":reach_score,
                  "relevance score":statuses_count+popularity_score+reach_score,
                  "company":company_name,
                  "sector":sector_map[query]}

            tweets_list5 += [body]

Tras el fiasco de FTX, JPMorgan hace la apuesta crypto más grande de su historia https://t.co/6KUjcJsynA -0.5106 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
@pwnwtfbbq Aka: crime, fraud, naked shorting via ETFs, hiding FTDs in SWAPS, rehypification, cellar boxing,… https://t.co/wVXPhmCr74 -0.8591 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitating the sexual abuse of young women and… -0.5859 negative
JPMorgan and Deutsche Bank sued by Epstein victims.

via @FinancialTimes 
#PrivateBanking #WealthManagement #bank… https://t.co/cjtgTG1ykJ -0.3182 negative
RT @Phil_Lewis_: JPMorgan Chase and Deutsche Bank have been accused of profiting from and facilitatin

RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
RT @GiftBetterUK: IT’S #CYBERMONDAY i!  🖥👾

And we have a £25 Vanilla MasterCard up for grabs! https://t.co/5hCc2XfEOV

For your chance to… 0.3786 positive
Want to land a job like "BizOps Engineer II" at Mastercard? Applicant tracking systems look for specific keywords.… https://t.co/AG2IU8OMtL 0.4215 positive
RT @voxto_amplify: A cross-ecosystem payment platform supporting #Web3 and Metaverse.  VISA, Mastercard or Google Pay, a variety of fiat cu… 0.3612 positive
RT @voxto_amplify: A cross-ecosystem payment platform supporting #Web3 and Metaverse.  VISA, Mastercard or Google Pay, a variety of fiat cu… 0.3612 positive
RT @voxto_amplify: A cross-ecosystem payment platform supporting #Web3 and Metaverse.  VISA, Mastercard or Google Pay, a variety of fiat cu… 0.3612 positive
RT @voxto_amplify: A cross-ecosystem payment platform support

Singapore will overtake Australia to have Asia’s highest share of millionaires in the adult population by 2030, acc… https://t.co/uHIgdDfzQA 0.296 positive
RT @theaugaf: Aramco Unit Hires HSBC, Citigroup for Riyadh Share Sale 
Riyadh November 27 2022: Saudi Aramco Base Oil Co., a unit of the st… 0.296 positive
Aramco Unit Hires HSBC, Citigroup for Riyadh Share Sale 
Riyadh November 27 2022: Saudi Aramco Base Oil Co., a unit… https://t.co/ie1eLOEhKH 0.296 positive
HSBC Holdings plc's quarterly revenue growth rate YoY of -9.20% ranked 3429 out of 5765 companies in our database.… https://t.co/4Pdv1SiYZV 0.3818 positive
HSBC Holdings plc's quarterly revenue growth rate YoY of -9.20% ranked 3429 out of 5765 companies in our database.… https://t.co/6VwOmKAJWo 0.3818 positive
RT @b20: Mark Tucker Group Chairman of @HSBC Holdings plc also emphasized the urgency of sustainable and renewable energy to accelerate dec… 0.2732 positive
RT @La_Resistencia1: 🇺🇲Citigroup Inc CN, HSBC Holdings Plc HSBA

RT @sophielouisecc: Extinction Rebellion women 'caused £100,000 of damage when they smashed Barclays HQ windows in Canary Wharf

They shoul… -0.5719 negative
RT @sophielouisecc: Extinction Rebellion women 'caused £100,000 of damage when they smashed Barclays HQ windows in Canary Wharf

They shoul… -0.5719 negative
RT @PetenShirl: Extinction Rebellion women 'caused £100,000 of damage when they smashed Barclays HQ windows in Canary Wharf demo'. She shou… -0.5719 negative
RT @MichaelAArouet: And just like that both global services and manufacturing PMI dropped below 50 👇 (Barclays via @patrick_saner) https://… 0.3612 positive
RT @sophielouisecc: Extinction Rebellion women 'caused £100,000 of damage when they smashed Barclays HQ windows in Canary Wharf

They shoul… -0.5719 negative
And just like that both global services and manufacturing PMI dropped below 50 👇 (Barclays via @patrick_saner) https://t.co/kwjmKsM0lv 0.3612 positive
This is an interesting result....civil recovery ordered in a

Just to mention some of the world bigger scandals in multiple industries and all of them with very strict regulatio… https://t.co/psEhkltcJL -0.4939 negative
RT @n194at: American Trans Air (ATA Airlines)
Lockheed L1011-500 N164AT
BUD/LHBP Budapest Ferihegy International Airport
Photo credit Feren… 0.3818 positive
Another shock result!  

American gun right group Safari Club International - which has shelled out £11 MILLION a Y… https://t.co/oxrPX8W27J -0.6476 negative
RT @JorgeDBL: Havana Consulting Group, Akerman, Akin Gump, Albright Stonebridge Group, American Maket Intelligence, Ankura Consulting, Carl… 0.4767 positive
RT @JorgeDBL: Havana Consulting Group, Akerman, Akin Gump, Albright Stonebridge Group, American Maket Intelligence, Ankura Consulting, Carl… 0.4767 positive
BLM, specifically the American chapter, not the international chapters that were duped, is nothing but a group of M… https://t.co/A0dalHDyRs -0.2263 negative
RT @FoundationZalmi: The group will spend the day at th

...

RT @MarianoGiustino: L'attrice #SoheilaGolestani ha pubblicato questo video sul suo profilo Instagram, insieme ad attori e attrici e profes… 0.34 positive
RT @GivingTuesdayIT: Da oggi solo tre giorni alla #giornatamondialedeldono.⏰
Hai tanti modi per partecipare e celebrare questa giornata e p… -0.34 negative
RT @bruno_simili: L’elogio dell’umiliazione come dispositivo pedagogico è particolarmente grave, visto che proviene dal ministro che dovreb… -0.3818 negative
RT @Gian59342003: Ranieri Guerra si avvale dell'immunità diplomatica dell' OMS x non essere giudicato a Bergamo.Ed ha fatto causa all'onest… 0.34 positive
RT @Yodobashi_X: ／
仕事も遊びもコレ1台👍
ゲーミングにも最適なPCモニター🎮
＼

美しい映像表現に加え、デュアルスピーカーも搭載✨
幅広い接続端子に対応した27インチフルHDモニター「#DELL S2723HC-R」⚡

フォロー＆RTで抽選で1名様にプレ… 0.3182 positive
@mauriziokomar @LPincia Il tuo legale 😂però ne hai di fantasia ... io fui sospeso poche ore dopo per le segnalazion… https://t.co/Gup5JpiJGA -0.2263 negative
RT @Yodobashi_X: ／
仕事も遊びもコレ1台👍
ゲーミングにも最適なPCモニター🎮
＼

美しい映像表現に加

𝐖𝐡𝐞𝐫𝐞 𝐭𝐨 𝐛𝐮𝐲 𝐒𝐡𝐨𝐩𝐢𝐟𝐲 𝐭𝐡𝐞𝐦𝐞𝐬
eCommerce is rising like never before, there was never a rage for online shopping like… https://t.co/OecuBqXDWh 0.6623 positive
Glad to see Shopify not bending their arm to the crybaby cancel mob. As a programmer myself I've always liked Shopi… https://t.co/jQhWmZs76Q 0.5859 positive
RT @libsoftiktok: Shopify CEO responds to pressure from the media to ban my shopify account 👑 https://t.co/4hmFzkZaoj -0.7003 negative
@OpenIntro @TomFitton @tobi @nandoodles @Shopify @libsoftiktok It must be hard for you to comprehend a persons clot… https://t.co/7CjzCmsgF9 -0.1027 negative
@DennyRun @MrAndyNgo @Shopify @nandoodles @nandood is getting a follow and a thank you.  It's about time.  The UK a… https://t.co/ioktgK0QjJ 0.3612 positive
RT @TymeBeach: Handmade bracelets available on our Shopify store!  We also have handmade holiday ornaments and Christmas stockings!
https:/… 0.508 positive
RT @libsoftiktok: Shopify CEO responds to pressure from the media to ban my shopi

🎯 #ETF Hunting by Keyword 🔑

🔎 Hey 🤖! Show me an #ETF investing in #Energy

🔮 ⚗ 🧪 ⚙ 👇

Ticker: $DTY

🔎Name: DTE Ene… https://t.co/KueINcntTw 0.3382 positive
Return triangle for DTE Energy Company #DTE @DTE_Energy https://t.co/Y6j3BLbvUL 0.2732 positive
Don't submit then quit. Apply for jobs like "Staff - ADMS Application Engineer DTE Electric" at DTE Energy, then lo… https://t.co/3qFN5npqEK 0.5574 positive
$DTE DTE Energy Company: Reference Price 109.15, 1 Year Prediction 123.91, Potential 14.0%, #DTE 0.2732 positive
Don't submit then quit. Apply for jobs like "Plant Manager" at DTE Energy, then look for people you might know at t… https://t.co/UlsvELfC0x 0.5574 positive
Return triangle for DTE Energy Company #DTE @DTE_Energy https://t.co/b624SdlYWA 0.2732 positive
Don't submit then quit. Apply for jobs like "Ops &amp; Maintenance Tech" at DTE Energy, then look for people you might… https://t.co/bUAGRyzsha 0.5574 positive
$DTE DTE Energy Company: Reference Price 109.15, 3 Year Predicti

The top ten stocks in the S&amp;P 500 by market capitalisation:

Apple (AAPL)
Microsoft (MSFT)
Amazon (AMZN)
Tesla (TSL… https://t.co/vkbmlGCo31 0.3612 positive
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
Working with Deloitte and Exxon Mobil is no longer a big deal because Tinubu did it. -0.296 negative
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil and gas company in history. Ye… 0.5859 positive
RT @PatRyanUC: Exxon Mobil reported $17B in profit last quarter alone. The highest profit reported by an oil a

RT @csMarksmaN: VALORANT BP GIVEAWAY!!
(instructions are in the photo itself)

#csMarksmaN🔥 https://t.co/Gv4FzruvHu -0.4559 negative
RT @beard_papa: ／
 ビアードパパ史上‼︎
 #ﾓｯﾄﾓｵｲﾓ 🍠
＼

ビアードパパの #焼いもシュー が #リニューアル して新登場🫰🏻

発売を記念してビアパパの特別割引券が当たるキャンペーンも開催🫶🏻

①@beard_papa をフォロー
②この投稿を… 0.4588 positive
RT @prim813: .⋅ ۵♡۵ ⋅. BP HYMM GA .⋅ ۵♡۵ ⋅.
- one winner of bp hymm 
- like retweet and follow @prim813 
- will end on december 2nd .⋅ ۵♡۵… 0.743 positive
RT @BadProsecutorBD: Details on BP Making Blu-ray (ver 0.1)

For now, actors commentaries confirmed. Additional contents &amp; more PO benefits… -0.6834 negative
@autumnjiwon id be fine getting two of these only if either bts or lyw would win aoty and bp is also in best perf but knowing msnake 💀 0.6597 positive
@AceLisa1997 @SUGOlSUGA @EmilysWORLD15 @_algernon_21 Defending bp but did they even defend blinks from haters 🤔 NO  NEVER   so next please -0.6417 negative
@jused13_bp и снова эта цепочка на талии 😍 0.4588 positive
RT @Koobidivines: Including

In [149]:
tweets_df = pd.DataFrame(tweets_list5)

In [150]:
tweets_df

,author,followers,date,message,hashtags,retweet,location,score,sentiment,likes,popularity score,reach score,relevance score,company,sector
0,mannilotrader,30,2022-11-28 08:14:16+00:00,"Tras el fiasco de FTX, JPMorgan hace la apuest...",None,0,BingX invitation code: 5UHMMQ,-0.5106,negative,254,254,-75,353,JPMorgan Chase & Co.,Financial Services
1,jaiye7777,2683,2022-11-28 08:09:18+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4234,,-0.5859,negative,59990,64224,4,87824,JPMorgan Chase & Co.,Financial Services
2,VHS_Jon,25,2022-11-28 08:09:07+00:00,"@pwnwtfbbq Aka: crime, fraud, naked shorting v...",None,0,,-0.8591,negative,5316,5316,-51,6438,JPMorgan Chase & Co.,Financial Services
3,sasha_sue87,13514,2022-11-28 08:08:30+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4234,"Iowa, USA",-0.5859,negative,19387,23621,3780,30202,JPMorgan Chase & Co.,Financial Services
4,LibertyLou4,1048,2022-11-28 08:08:09+00:00,RT @Phil_Lewis_: JPMorgan Chase and Deutsche B...,None,4234,"Orlando, FL",-0.5859,negative,13885,18119,-1846,39705,JPMorgan Chase & Co.,Financial Services
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,CasanareonlineC,104,2022-11-23 02:19:56+00:00,#Fiestas ||Empresas petroleras como ECOPETROL ...,Fiestas,0,"Yopal, Colombia",0.5574,positive,4,4,-87,2934,ECOPETROL_SA,Energy
1079,dinsightsid,573,2022-11-24 08:00:23+00:00,SKK Migas reveals that Chinese oil and gas com...,None,0,,0.4019,positive,90,90,571,2915,PetroChina,Energy
1080,Tickeron,17916,2022-11-28 07:27:35+00:00,$SLB vs. $TS: what will be the best investment...,Schlumberger,0,"Sunnyvale, CA",0.6369,positive,48,48,17180,2129248,Schlumberger,Energy
1081,israel_amara,25,2022-11-28 04:19:55+00:00,zaman pirler bulunmuyor salonlarda. Her gun bi...,None,45,,-0.3400,negative,224,269,-17,1205,Schlumberger,Energy


In [151]:
len(tweets_df["company"].unique())

29

In [152]:
tweets_df["company"].value_counts()

exxonmobil              92
Mastercard              89
Barclays                80
autodesk                72
Atlassian               63
amazon                  58
JPMorgan Chase & Co.    57
Shopify                 56
Shell                   53
Apple                   52
VMware                  51
LloydsBank              47
nokia                   43
BankofAmerica           40
bp_plc                  39
GoldmanSachs            37
Adobe                   33
Dell                    28
DTE_Energy              24
Broadcom                13
AIGinsurance            11
AmericanExpress         11
HSBC                    10
cibc                     9
HessCorporation          5
DukeEnergy               4
Schlumberger             3
ECOPETROL_SA             2
PetroChina               1
Name: company, dtype: int64

In [153]:
tweets_df.to_csv(r"C:\Users\Damilola\Desktop\MSIA Courses\Second Year\First Semester\IAL 620\Project\Twitter Sentiments\sentiment_tweets.csv", index=False)

In [154]:
data._json["user"]["favourites_count"]

16588